<a href="https://colab.research.google.com/github/sunshineluyao/UTXO/blob/main/UTXO_data_analysis_Task_1_Append1_2020_11_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1: Calculate Daily UTXO: First Append Notebook

## Import Data from Google Drive and Data Wrangling

In [163]:
import numpy as np
import pandas as pd
import datetime

In [164]:
!pip install chart_studio

In [165]:
# Importing drive method from colab for accessing google drive
from google.colab import drive

# Mounting drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
import numpy as np
import pandas as pd
df_UTXO_Old=pd.read_csv('/content/drive/My Drive/UTXO/df_UTXO.csv',index_col='Unnamed: 0')
df_UTXO_Old.tail()

,date,UTXO_newborn,UTXO_dead,Net_New,UTXO_Cum
723,2010-12-27,58656.806641,50774.316641,7882.490000,4.211390e+06
724,2010-12-28,30374.153958,22945.883958,7428.270000,4.218819e+06
725,2010-12-29,86575.966836,78932.732582,7643.234254,4.226462e+06
726,2010-12-30,38568.962386,30381.682386,8187.280000,4.234649e+06
727,2010-12-31,29212.393351,21489.893351,7722.500000,4.242372e+06


In [167]:
df=pd.read_csv('/content/drive/My Drive/UTXO/joint_2011.csv',index_col='Unnamed: 0')
df.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,value,block_date,spent_block_date
15563,957000000,2011-01-01,2011-01-02
46791,29000000,2011-01-01,2011-01-02
64685,33895000000,2011-01-01,2011-01-02
94558,126000000,2011-01-01,2011-01-02
113788,4552000000,2011-01-01,2011-01-02


In [168]:
# Generate the UTXO value in bitcoin unit, which = $value/10^{8}$
df['UTXO'] = df['value']*10**(-8)
df = df.reset_index(drop=True)
df = df.drop(['value'], axis = 1)
df.tail()

,block_date,spent_block_date,UTXO
2057628,2011-12-31,2019-12-17,0.713323
2057629,2011-12-31,2020-03-21,0.711767
2057630,2011-12-31,2020-04-19,1.030000
2057631,2011-12-31,2020-04-26,0.001000
2057632,2011-12-31,2020-05-11,0.001000


In [169]:
# Change the block_date and spent_block_date to datatime object
df['block_date'] = pd.to_datetime(df['block_date'], format='%Y-%m-%d')
df['spent_block_date'] = pd.to_datetime(df['spent_block_date'], format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2057633 entries, 0 to 2057632
Data columns (total 3 columns):
 #   Column            Dtype         
---  ------            -----         
 0   block_date        datetime64[ns]
 1   spent_block_date  datetime64[ns]
 2   UTXO              float64       
dtypes: datetime64[ns](2), float64(1)
memory usage: 47.1 MB


In [170]:
start =pd.to_datetime(df['block_date'].iloc[0], format='%Y-%m-%d')

In [171]:
start

Timestamp('2011-01-01 00:00:00')

In [172]:
end = pd.to_datetime(df['block_date'].iloc[-1], format='%Y-%m-%d')

In [173]:
end

Timestamp('2011-12-31 00:00:00')

# Repeat similar treatments on the Appending datasets

# For each date, calculate the total UTXO generated on that date

In [174]:
#sort the data by blockdate
df = df.sort_values(by = 'block_date')
df.reset_index(drop = True, inplace = True)
df.tail()

,block_date,spent_block_date,UTXO
2057628,2011-12-31,2012-01-03,0.311839
2057629,2011-12-31,2012-01-03,0.600000
2057630,2011-12-31,2012-01-03,0.232502
2057631,2011-12-31,2012-01-03,4.400000
2057632,2011-12-31,2020-05-11,0.001000


In [175]:
### use reset_index() to keep the grouping variable as a column
df_newborn = df.groupby(df['block_date']).sum()['UTXO'].reset_index()
df_newborn.rename(columns={"UTXO": "UTXO_newborn"}, inplace=True)
df_newborn.tail()

,block_date,UTXO_newborn
360,2011-12-27,153975.918967
361,2011-12-28,137154.743214
362,2011-12-29,164100.848351
363,2011-12-30,131486.582052
364,2011-12-31,132844.230895


## For each date, calculate the total UTXO spent on that date

In [176]:
#sort the data by spent_block_date
df = df.sort_values(by = 'spent_block_date')
df.reset_index(drop = True, inplace = True)
df.head()

,block_date,spent_block_date,UTXO
0,2011-01-01,2011-01-02,9.57
1,2011-01-01,2011-01-02,25.00
2,2011-01-01,2011-01-02,5.01
3,2011-01-01,2011-01-02,47.43
4,2011-01-01,2011-01-02,38.55


In [177]:
### use reset_index() to keep the grouping variable as a column
df_dead = df.groupby(df['spent_block_date']).sum()['UTXO'].reset_index()
df_dead.rename(columns={"UTXO": "UTXO_dead"}, inplace=True)
df_dead.tail()

,spent_block_date,UTXO_dead
3415,2020-10-10,0.999589
3416,2020-10-13,0.001000
3417,2020-10-14,16.479674
3418,2020-10-15,1.675516
3419,2020-10-16,10.770000


# Append the old dead dataframe and calculate UTXO for the new time range

In [178]:
df_dead_old=pd.read_csv('/content/drive/My Drive/UTXO/df_dead1.csv',index_col='Unnamed: 0')
df_dead_old.head()

,spent_block_date,UTXO_dead
468,2011-01-01,11299.459153
469,2011-01-02,19522.082666
470,2011-01-03,18643.645624
471,2011-01-04,5385.997750
472,2011-01-05,12522.985279


In [179]:
df_dead=df_dead.append(df_dead_old, ignore_index=True)
df_dead.tail()

,spent_block_date,UTXO_dead
5067,2020-09-22,50.0
5068,2020-10-01,50.0
5069,2020-10-11,1050.0
5070,2020-10-14,50.0
5071,2020-10-17,50.0


In [180]:
### use reset_index() to keep the grouping variable as a column
df_dead = df_dead.groupby(df_dead['spent_block_date']).sum()['UTXO_dead'].reset_index()
df_dead.tail()

,spent_block_date,UTXO_dead
5067,2020-09-22,50.0
5068,2020-10-01,50.0
5069,2020-10-11,1050.0
5070,2020-10-14,50.0
5071,2020-10-17,50.0


 ### Merge the two dataframe to get the time series for newborn and dead UTXO for each date from 2011-01-01 to 2011-12-31

In [181]:
size=np.size(pd.date_range(start=start, end=end))

In [182]:
df_UTXO = pd.DataFrame(np.zeros((size, 1)))
df_UTXO.columns=['date']
df_UTXO['date']=pd.date_range(start=start, end=end)
df_UTXO.tail()

,date
360,2011-12-27
361,2011-12-28
362,2011-12-29
363,2011-12-30
364,2011-12-31


In [183]:
### merge
df_UTXO=df_UTXO.merge(df_newborn, how='left',left_on='date',right_on='block_date')
df_UTXO.head()

,date,block_date,UTXO_newborn
0,2011-01-01,2011-01-01,20145.909153
1,2011-01-02,2011-01-02,31900.642666
2,2011-01-03,2011-01-03,34039.113301
3,2011-01-04,2011-01-04,17087.245427
4,2011-01-05,2011-01-05,25500.238467


In [184]:
### merge
df_UTXO=df_UTXO.merge(df_dead, how='left',left_on='date',right_on='spent_block_date')
df_UTXO.head()

,date,block_date,UTXO_newborn,spent_block_date,UTXO_dead
0,2011-01-01,2011-01-01,20145.909153,2011-01-01,11299.459153
1,2011-01-02,2011-01-02,31900.642666,2011-01-02,4151.240000
2,2011-01-02,2011-01-02,31900.642666,2011-01-02,19522.082666
3,2011-01-03,2011-01-03,34039.113301,2011-01-03,8113.997677
4,2011-01-03,2011-01-03,34039.113301,2011-01-03,18643.645624


In [185]:
df_UTXO = df_UTXO.drop(['block_date','spent_block_date'], axis = 1)

In [186]:
df_UTXO=df_UTXO[['date','UTXO_newborn','UTXO_dead']].fillna(0)
df_UTXO.head()

,date,UTXO_newborn,UTXO_dead
0,2011-01-01,20145.909153,11299.459153
1,2011-01-02,31900.642666,4151.240000
2,2011-01-02,31900.642666,19522.082666
3,2011-01-03,34039.113301,8113.997677
4,2011-01-03,34039.113301,18643.645624


Calculate the Daily Net new UTXO =UTXO_newborn-UTXO_dead

In [187]:
df_UTXO['Net_New']=df_UTXO['UTXO_newborn']-df_UTXO['UTXO_dead']
df_UTXO.head()

,date,UTXO_newborn,UTXO_dead,Net_New
0,2011-01-01,20145.909153,11299.459153,8846.450000
1,2011-01-02,31900.642666,4151.240000,27749.402666
2,2011-01-02,31900.642666,19522.082666,12378.560000
3,2011-01-03,34039.113301,8113.997677,25925.115624
4,2011-01-03,34039.113301,18643.645624,15395.467677


Calculate the Accumulated Sum of Net new UTXO to get the total UTXO not spent for each date, **Note** This column is incorrect and needs to be adjusted later. The whole columns needs to add the last row of this column in the old df_UTXO

In [188]:
df_UTXO['UTXO_Cum']=df_UTXO['Net_New'].cumsum()
df_UTXO.head()

,date,UTXO_newborn,UTXO_dead,Net_New,UTXO_Cum
0,2011-01-01,20145.909153,11299.459153,8846.450000,8846.450000
1,2011-01-02,31900.642666,4151.240000,27749.402666,36595.852666
2,2011-01-02,31900.642666,19522.082666,12378.560000,48974.412666
3,2011-01-03,34039.113301,8113.997677,25925.115624,74899.528289
4,2011-01-03,34039.113301,18643.645624,15395.467677,90294.995967


In [189]:
df_UTXO['UTXO_Cum']= df_UTXO['UTXO_Cum']+df_UTXO_Old['UTXO_Cum'].iloc[-1]
df_UTXO.head()

,date,UTXO_newborn,UTXO_dead,Net_New,UTXO_Cum
0,2011-01-01,20145.909153,11299.459153,8846.450000,4.251218e+06
1,2011-01-02,31900.642666,4151.240000,27749.402666,4.278968e+06
2,2011-01-02,31900.642666,19522.082666,12378.560000,4.291346e+06
3,2011-01-03,34039.113301,8113.997677,25925.115624,4.317271e+06
4,2011-01-03,34039.113301,18643.645624,15395.467677,4.332667e+06


In [190]:
df_UTXO.tail()

,date,UTXO_newborn,UTXO_dead,Net_New,UTXO_Cum
708,2011-12-29,164100.848351,405.050000,163695.798351,5.708615e+07
709,2011-12-30,131486.582052,123989.170803,7497.411249,5.709365e+07
710,2011-12-30,131486.582052,650.050000,130836.532052,5.722448e+07
711,2011-12-31,132844.230895,125170.490523,7673.740372,5.723216e+07
712,2011-12-31,132844.230895,305.100000,132539.130895,5.736470e+07


# Append the New UTXO to the Old UTXO Dataframe

In [196]:
df_UTXO=df_UTXO_Old.append(df_UTXO, ignore_index=True)
df_UTXO['date'] = pd.to_datetime(df_UTXO['date'], format='%Y-%m-%d')
df_UTXO.tail()

,date,UTXO_newborn,UTXO_dead,Net_New,UTXO_Cum
5076,2011-12-29,164100.848351,405.050000,163695.798351,5.708615e+07
5077,2011-12-30,131486.582052,123989.170803,7497.411249,5.709365e+07
5078,2011-12-30,131486.582052,650.050000,130836.532052,5.722448e+07
5079,2011-12-31,132844.230895,125170.490523,7673.740372,5.723216e+07
5080,2011-12-31,132844.230895,305.100000,132539.130895,5.736470e+07


In [197]:
df_UTXO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5081 entries, 0 to 5080
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          5081 non-null   datetime64[ns]
 1   UTXO_newborn  5081 non-null   float64       
 2   UTXO_dead     5081 non-null   float64       
 3   Net_New       5081 non-null   float64       
 4   UTXO_Cum      5081 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 198.6 KB


# Generate the Figures

In [198]:
import plotly.graph_objects as go
from plotly.offline import iplot

# Create traces
fig0 = go.Figure()
fig0.add_trace(go.Scatter(x=df_UTXO.date, y=df_UTXO.UTXO_newborn,
                    mode='lines',
                    name='UTXO Newborn'))

fig0.add_trace(go.Scatter(x=df_UTXO.date, y=df_UTXO.UTXO_dead,
                    mode='lines+markers',
                    name='UTXO Dead'))

fig0.add_trace(go.Scatter(x=df_UTXO.date, y=df_UTXO.Net_New,
                    mode='markers', name='UTXO Netnew'))
fig0.update_layout(title='Daily Newborn and Dead UTXO',
                   xaxis_title='Date',
                   yaxis_title='UTXO')
fig0.update_yaxes(type="log")


# Add range slider
fig0.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

iplot(fig0)

In [199]:
import plotly.graph_objects as go
from plotly.offline import iplot

# Create traces
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=df_UTXO.date, y=df_UTXO.UTXO_Cum,
                    mode='markers', name='UTXO Cum'))
fig1.update_layout(title='The Accumulated UTXO',
                   xaxis_title='Date',
                   yaxis_title='UTXO')
# Add range slider
fig1.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

iplot(fig1)

# Save the datasets

## 1. The Datasets for Final Products

In [200]:
df_UTXO.to_csv('/content/drive/My Drive/UTXO/df_UTXO1.csv')

## 2. The Datasets for calculating Final Products in the future dates

In [201]:
df_dead.tail()

,spent_block_date,UTXO_dead
5067,2020-09-22,50.0
5068,2020-10-01,50.0
5069,2020-10-11,1050.0
5070,2020-10-14,50.0
5071,2020-10-17,50.0


In [202]:
df_dead=df_dead[df_dead['spent_block_date']>end]

In [203]:
df_dead.to_csv('/content/drive/My Drive/UTXO/df_dead1.csv')